# Objetivo 2.1

# Autores
David Rodríguez , Alejandro Meza

# Índice de la práctica

- [1.Introducción](#1.Introducción)
- [2.Creación de datos](#2.Creación-de-datos)
- [3.Creación de Modelo de Predicción](#3.Creación-de-Modelo-de-Predicción)

In [2]:
import cv2
import glob
import skimage


#Librerias contornos 
import numpy as np
from skimage.io import imread, imshow
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.feature import hog
from skimage import data, exposure

#Machine Learning
from scipy.optimize import minimize
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV
from sklearn import svm

In [3]:
import cv2
import glob
import skimage


#Librerias contornos 
import numpy as np
from skimage.io import imread, imshow
from skimage.filters import prewitt_h,prewitt_v
from skimage import data, io
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.filters import threshold_otsu 
from skimage.feature import hog
from skimage import data, exposure

#Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris #dataset de prueba
from sklearn.preprocessing import OneHotEncoder
from scipy.optimize import minimize
import sklearn.metrics as metrics
from sklearn.model_selection import GridSearchCV
from sklearn import svm

#Librerias para preparación de datos
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 1.Introducción

Para cumplir el objetivo 2, lo hemos dividido en tareas:

||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

# 2.Creación de datos

Se va a crear la matriz X, que es la que contiene las imágenes (se crea un dataset). Creamos metodos que nos ayudaran a conseguir nuestro objetivo.

In [4]:
'''
Descripcion: diccionario con los posibles valores de las imagenes, en relacion con el nombre de las carpetas.
Entrada: -
Salida: diccionario cuyas claves son el nombre de las carpetas, y los valores son el tipo de signals que contiene.
'''
def devolverDiccionarioEtiquetasSign():
    
    diccionarioValores = {}
    diccionarioValores['00000'] = 'VelocidadMaxima20'
    diccionarioValores['00001'] = 'VelocidadMaxima30'
    diccionarioValores['00002'] = 'VelocidadMaxima50'
    diccionarioValores['00003'] = 'VelocidadMaxima60'
    diccionarioValores['00004'] = 'VelocidadMaxima70'
    diccionarioValores['00005'] = 'VelocidadMaxima80'
    diccionarioValores['00006'] = 'FinVelocidadMaxima80'
    diccionarioValores['00007'] = 'VelocidadMaxima100'
    diccionarioValores['00008'] = 'VelocidadMaxima120'
    diccionarioValores['00009'] = 'AdelantamientoProhibido'
    diccionarioValores['00010'] = 'AdelantamientoProhibidoParaCamiones'

    diccionarioValores['00011'] = 'InterseccionConPrioridad'
    diccionarioValores['00012'] = 'CalzadaConPrioridad'
    diccionarioValores['00013'] = 'CedaElPaso'
    diccionarioValores['00014'] = 'DetencionObligatoria'
    diccionarioValores['00015'] = 'CirculacionProhibida'
    diccionarioValores['00016'] = 'EntradaProhibidaAVehiculosMercancias'
    diccionarioValores['00017'] = 'EntradaProhibida'
    diccionarioValores['00018'] = 'OtrosPeligros'
    diccionarioValores['00019'] = 'CurvaPeligrosaHaciaLaIzquierda'
    diccionarioValores['00020'] = 'CurvaPeligrosaHaciaLaDerecha'

    diccionarioValores['00021'] = 'CurvasPeligrosasHaciaLaIzquierda'
    diccionarioValores['00022'] = 'PerfilIrregular'
    diccionarioValores['00023'] = 'PavimentoDeslizante'
    diccionarioValores['00024'] = 'EstrechamientoCalzadaPorDerecha'
    diccionarioValores['00025'] = 'Obras'
    diccionarioValores['00026'] = 'Semaforos'
    diccionarioValores['00027'] = 'PasoDePeatones'
    diccionarioValores['00028'] = 'Kids'
    diccionarioValores['00029'] = 'EntradaProhibidasCiclos'
    diccionarioValores['00030'] = 'PavimentoDeslizanteNieveHielo'

    diccionarioValores['00031'] = 'PasoDeAnimalesEnLibertad'
    diccionarioValores['00032'] = 'FinDeProhibiciones'
    diccionarioValores['00033'] = 'SentidoObligatorioDerecha'
    diccionarioValores['00034'] = 'SentidoObligatorioIzquierda'
    diccionarioValores['00035'] = 'SentidoObligatorio'
    diccionarioValores['00036'] = 'DirPermitidasRectoYDerecha'
    diccionarioValores['00037'] = 'DirPermitidasRectoEIzquierda'
    diccionarioValores['00038'] = 'PasoObligatorioDerecha'
    diccionarioValores['00039'] = 'PasoObligatorioizquierda'
    diccionarioValores['00040'] = 'InterseccionSentidoObligatorioGiratorio'


    diccionarioValores['00041'] = 'FinProhibicionAdelantamiento'
    diccionarioValores['00042'] = 'FinProhibicionAdelantamientoCamiones'
    
    return diccionarioValores

In [5]:
'''
Descripcion: diccionario con los posibles valores de las imagenes, en relacion con el nombre de las carpetas.
Entrada: -
Salida: diccionario cuyas claves son el nombre de las carpetas, y los valores son el tipo de signals que contiene.
'''
def devolverDiccionarioEtiquetasNoSign():
    
    diccionarioValores = {}
    diccionarioValores['000'] = 'Animal'
    diccionarioValores['001'] = 'Carretera'
    diccionarioValores['002'] = 'Cielo'
    diccionarioValores['003'] = 'VehiculoCoche'
    diccionarioValores['004'] = 'Arbol'
    diccionarioValores['005'] = 'Persona'
    diccionarioValores['006'] = 'Edificio'
    diccionarioValores['007'] = 'VehiculoMoto'
    diccionarioValores['008'] = 'Semaforo'
    diccionarioValores['009'] = 'Basura'
    diccionarioValores['010'] = 'Farola'

    diccionarioValores['011'] = 'Estatua'
    diccionarioValores['012'] = 'PasoDeZebra'
    diccionarioValores['013'] = 'Carteles'
    diccionarioValores['014'] = 'Ventana'
    diccionarioValores['015'] = 'Paisaje'
    diccionarioValores['016'] = 'Mix'
    diccionarioValores['017'] = 'All'
    diccionarioValores['018'] = 'Prueba'
    
    return diccionarioValores

Probemos ahora el código....

In [6]:
d =  devolverDiccionarioEtiquetasNoSign()

for k,v in d.items():
    print(v)

Animal
Carretera
Cielo
VehiculoCoche
Arbol
Persona
Edificio
VehiculoMoto
Semaforo
Basura
Farola
Estatua
PasoDeZebra
Carteles
Ventana
Paisaje
Mix
All
Prueba


Creamos un diccionario para almacenar todas las fotos, relacionadas a cada elemento diferente de señal

In [7]:
'''
Descripcion: diccionario con los posibles valores de las imagenes, en relacion con el nombre de las carpetas.
Entrada: -
Salida: diccionario cuyas claves son el nombre de las carpetas, y los valores son las imagenes de las signals
'''
def crearDiccionarioDatosSign():
    trafic_signals = {}
    #para cada tipo de señal creamos una lista con sus ejemplos, y se la asignamos a su etiqueta de nombre de carpeta
    for i in range(0,43):
        if i<10:
            folder = 'Dataset_traffic_sign/0000'+str(i)+'/*.png'
            tipo = '0000'+str(i)
        else:
            folder = 'Dataset_traffic_sign/000'+str(i)+'/*.png'
            tipo = '000'+str(i)
        
        images = [cv2.imread(file,0) for file in glob.glob(folder)]
        images_mod = [] 
        for im in images:
            #se aplica el resize
            images_mod.append(cv2.resize(im, (100, 100), interpolation = cv2.INTER_NEAREST))

        trafic_signals[tipo] = images_mod
    return trafic_signals

In [8]:
'''
Descripcion: diccionario con los posibles valores de las imagenes, en relacion con el nombre de las carpetas.
Entrada: -
Salida: diccionario cuyas claves son el nombre de las carpetas, y los valores los elementos que no son signals
'''
def crearDiccionarioDatosNoSign():
    noSignals = {} #diccionario para almacenar todo lo que no es sinal
    #para cada tipo de elemento que no es señal creamos una lista con sus ejemplos
    # y se la asignamos a su etiqueta de nombre de carpeta
    for i in range(0,18): #iteramos sobre el numero de carptas
        if i<10:
            folder = 'Dataset_no_traffic_sign/00'+str(i)+'/*.jpg'
            tipo = '00'+str(i)
        else:
            folder = 'Dataset_no_traffic_sign/0'+str(i)+'/*.jpg'
            tipo = '0'+str(i)

        images = [cv2.imread(file,0) for file in glob.glob(folder)]
        images_mod = [] 
        for im in images:
            #se aplica el resize
            images_mod.append(cv2.resize(im, (100, 100), interpolation = cv2.INTER_NEAREST))

        noSignals[tipo] = images_mod
    return noSignals

In [9]:
'''
Descripcion: -
Entrada: -
Salida: lista con ejemplos de calles
'''
def crearListaCalle():
    folder = 'Dataset_images/*.jpg'
    images = [cv2.imread(file) for file in glob.glob(folder)]
    images_mod = [] 
    for im in images:
        #se aplica el resize
        images_mod.append(cv2.resize(im, (100, 100), interpolation = cv2.INTER_NEAREST))

    images_mod
    return images_mod

Creamos funciones para transformar imagenes en descriptores con HOG.

In [10]:
# HOG
def hogMaker(images00000_mod):
    fd, hog_image = hog(images00000_mod, orientations=8, pixels_per_cell=(16, 16),
                        cells_per_block=(1, 1), visualize=True)
    return fd

Cogemos las imagenes y las separamos en train y test, y decimos a que clase pertenecen*

In [11]:
'''
Descripcion: Obtencion de train y test a partir de un conjunto de imágenes
Entrada: trafic_signals, signals_types
Salida: imagenesTrain,imagenesTest,ytrain,ytest
'''
def separaTrainTest(trafic_signals,signals_types):
    imagenesTrain = None
    imagenesTest = None
    ytrain = None
    ytest = None
    
    for signal,images in trafic_signals.items():
        
        aux_y_train = []
        aux_y_test = []
        
        np_im = np.array(images)
        n_train = int(0.6*np_im.shape[0])
        n_test = np_im.shape[0]-n_train
        
        aux_train = np_im[:n_train]
        aux_test = np_im[n_train:]
        
        for i in range(n_train):
            aux_y_train.append(signals_types[signal])
        for i in range(n_test):
            aux_y_test.append(signals_types[signal])
        
        aux_y_train = np.array(aux_y_train).reshape(-1,1)
        aux_y_test = np.array(aux_y_test).reshape(-1,1)
        
        if imagenesTrain is None:
            imagenesTrain = aux_train
            ytrain = aux_y_train
            imagenesTest = aux_test
            ytest = aux_y_test
        else:
            imagenesTrain = np.concatenate((imagenesTrain,aux_train),axis=0)
            ytrain =  np.vstack((ytrain,aux_y_train))
            imagenesTest = np.concatenate((imagenesTest,aux_test),axis=0)
            ytest = np.vstack((ytest,aux_y_test))
        
    return imagenesTrain,imagenesTest,ytrain,ytest

Creamos las caracteristicas con los datos que nos den y sus tipos*

In [12]:

def creaMatrizCaracteristicas(imagenes,carac_type, y=None ):
    X = None
    if y is not None:
        y_nueva = []
        
    for i in range(imagenes.shape[0]):
        if carac_type=='hog':
            d = hogMaker(imagenes[i])
        else: #en caso de recibir un valor que no es valido
            try:
                raise ValueError
            except ValueError:
                print("valor incorrecto")
        
        if d is not None:
            if y is not None:
                y_nueva.append(y[i])
            if X is None:
                X = d
            else:
                X = np.vstack((X,d))
    if y is not None:
        return X, np.array(y_nueva)
    else:
        return X

In [13]:
elem_signals = crearDiccionarioDatosSign() #creacion del diccionario de datos
signals_types = devolverDiccionarioEtiquetasSign() #creacion de diccionario que contiene nombre de carpetas y tipos de signals

elem_no_signals = crearDiccionarioDatosNoSign() #creacion del diccionario de datos
no_signals_types = devolverDiccionarioEtiquetasNoSign() #creacion de diccionario que contiene nombre de carpetas y tipos de signalcarac_type = 'hog' #el tipo elegido para los descriptores va a ser Hog
carac_type = 'hog'

In [14]:
len(elem_no_signals['017'])


9823

Separamos las señales y las no señales en conjuntos para TRAIN y TEST

In [15]:
signal_Train,signal_Test,signal_ytrain,signal_ytest = separaTrainTest(elem_signals,signals_types)

In [16]:
no_signal_Train,no_signal_Test,no_signal_ytrain,no_signal_ytest = separaTrainTest(elem_no_signals,no_signals_types)

In [17]:
imagenesTrain = np.concatenate((signal_Train, no_signal_Train))
imagenesTest = np.concatenate((signal_Test, no_signal_Test))

ytrain = np.vstack((signal_ytrain,no_signal_ytrain))
ytest = np.vstack((signal_ytest,no_signal_ytest))

Xtrain, Ytrain = creaMatrizCaracteristicas(imagenesTrain,carac_type,ytrain)
Xtest, Ytest = creaMatrizCaracteristicas(imagenesTest,carac_type,ytest)

In [18]:
def filtro_suavizado(imagen, mascara):
    a,b = mascara.shape
    a = (a//2)
    b = (b//2)
    aux = np.zeros_like(imagen)

    #Se crea la imagen ampliada 
    imagen_ampliada = cv2.copyMakeBorder(imagen,a,a,b,b,cv2.BORDER_REPLICATE)

    #Dimensiones de la imagen ampliada
    r,c = imagen_ampliada.shape

    for i in range(a,r-a): #en el bucle no iteras sobre las filas que has ampliado
        for j in range(b,c-b): #en el bucle no iteras sobre las columnas que has ampliado
            vecindad= imagen_ampliada[i-a:i+a+1, j-b: j+b+1]
            aux[i-a,j-b] = np.sum(vecindad * mascara)

    return np.uint8(aux)

n = 3 #tamaño del filtro
sigma = 3 #desviación de la gaussiana
mask = cv2.getGaussianKernel(n, sigma)*cv2.getGaussianKernel(n, sigma).T

imagenesTrain_GK = np.zeros(imagenesTrain.shape)
for i in range(imagenesTrain.shape[0]):
    imagenesTrain_GK[i] = filtro_suavizado(imagenesTrain[i], mask)

imagenesTest_GK = np.zeros(imagenesTest.shape)
for i in range(imagenesTest.shape[0]):
    imagenesTest_GK[i] = filtro_suavizado(imagenesTest[i], mask)
    
Xtrain, Ytrain = creaMatrizCaracteristicas(imagenesTrain_GK,carac_type,ytrain)
Xtest, Ytest = creaMatrizCaracteristicas(imagenesTest_GK,carac_type,ytest)

In [19]:
signal_Train.shape,signal_Test.shape,signal_ytrain.shape,signal_ytest.shape

((739, 100, 100), (525, 100, 100), (739, 1), (525, 1))

In [20]:
no_signal_Train.shape,no_signal_Test.shape,no_signal_ytrain.shape,no_signal_ytest.shape

((7962, 100, 100), (5320, 100, 100), (7962, 1), (5320, 1))

In [21]:
Xtrain.shape, Ytrain.shape,Xtest.shape, Ytest.shape

((8701, 288), (8701, 1), (5845, 288), (5845, 1))

# 3.Creación de Modelo de Predicción

A continuación se implementarán dos modelos de ML como son regresión logísitica y SVM para dar solución al problema.

In [22]:
def transformarY_SignNoSign(y, dicNoSign):
    Y = np.ones(y.shape[0]).reshape(-1,1)
    
    for signal,nombre in dicNoSign.items():
        for i in range(Y.size):
            if y[i]==nombre:
                Y[i] = 0
    Yfinal = []
    for i in range(Y.size):
        if Y[i] == 1:
            Yfinal.append('Signal')
        else:
            Yfinal.append('No Signal')
    return Yfinal

In [23]:
Ytrain_T = transformarY_SignNoSign(Ytrain, no_signals_types)
Ytest_T = transformarY_SignNoSign(Ytest, no_signals_types)

In [24]:
Ytrain_T = np.array(Ytrain_T)
Ytest_T = np.array(Ytest_T)

In [25]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},]
svc = svm.SVC()
clf = GridSearchCV(svc, param_grid)
clf.fit(Xtrain,Ytrain_T.ravel())
y_pred = clf.predict(Xtest)
print("Classification report for - \n{}:\n{}\n".format(clf, metrics.classification_report(Ytest_T.ravel(), y_pred)))

Classification report for - 
GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}]):
              precision    recall  f1-score   support

   No Signal       1.00      1.00      1.00      5320
      Signal       0.99      0.95      0.97       525

    accuracy                           1.00      5845
   macro avg       0.99      0.98      0.99      5845
weighted avg       1.00      1.00      1.00      5845




In [26]:
def predecirImagenSVM(imagen,carac_type,clf):
#     transformar imagen en des
    d = creaMatrizCaracteristicas(np.array([imagen]), carac_type)
    d = np.array(d).reshape(1,-1)
    return clf.predict(d)

In [27]:
#pequeña celda para interactuar con el clasificador

caracter = input("¿Desea probar el clasificador? (si/no)")
nDeAciertos = 0
nDeSignalsEjecutadas = 0
if caracter == "no":
    print("Hasta la proxima!!")
elif caracter == "si":
    while(caracter == "si"):
        nDeSignalsEjecutadas+=1
        numeroImagen = (int)(input("Introduce un numero de imagen para probar: "))
        print(f"Tu numero de imagen para probar es la numero {numeroImagen}")
        res = predecirImagenSVM(imagenesTest[numeroImagen],carac_type,clf)
        plt.figure()
        plt.imshow(imagenesTest[numeroImagen])
        lab = 'Precision 95%\n'+ res[0]
        plt.xlabel(lab)
        plt.show()
        if Ytest_T[numeroImagen]==res:
            nDeAciertos+=1
        print(f"Precision hasta el momento: {(nDeAciertos/nDeSignalsEjecutadas)*100}%\n")
        caracter = input("¿Desea seguir probando el clasificador? (si/no)")
        if caracter == "no":
            print("Hasta la proxima!!")
            break

¿Desea probar el clasificador? (si/no)no
Hasta la proxima!!


Para finalizar, vemos que tenemmos una gran tasa de aciertos, para predecir entre señales y no señales.

# Objetivo 2.2

Para buscar en la imagen una señal, vamos a recorrer la imagen con una ventana la cual va a variar el tamaño en busca de las señales

In [28]:
import argparse
import time
import cv2

from skimage.transform.pyramids import pyramid_gaussian

In [29]:
def sliding_window(image, entreVentanas, windowSize):
    # slide a window across the image
    for j in range(0, image.shape[0], entreVentanas):
        for i in range(0, image.shape[1], entreVentanas):
            # yield the current window
            yield (i, j, image[j:j + windowSize[1], i:i + windowSize[0]])

Paso a paso vamos a ver como recorre la imagen

Cogemos una imagen

In [ ]:
a = cv2.imread('00033.jpg',0)
b = cv2.imread('00033.jpg')

Con un tamaño de ventana cuadrada igual al menor lado de la imagen

In [ ]:
if a.shape[0]<a.shape[1]:
    winH = int(a.shape[0])
else:
    winH = int(a.shape[1])
winW = winH
step = int(winW/2)
# print(winW,winH,step)
result = []
while winW>30 and winH>30:
    # loop over the sliding window for each layer of the pyramid
    for (x, y, window) in sliding_window(a, step, windowSize=(winW, winH)):
        # if the window does not meet our desired window size, ignore it
        if x+winW<=a.shape[1] and y+winH<=a.shape[0]:
            res = predecirImagenSVM(cv2.resize(window, (100, 100), interpolation = cv2.INTER_NEAREST),carac_type,clf)
#             clone = a.copy()
#             cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
#             cv2.imshow("Window", clone)
            if res=='Signal':
#                 cv2.waitKey(0)
                result.append(np.array([x,y,winW,winH]))
#             else:
#                 cv2.waitKey(1)
#                 time.sleep(0.001)
#     cv2.destroyAllWindows()
    winW = int(winW/1.5)
    winH = winW
    step = int(winW/2)
#     print(winW,winH)

Aplicar SVM

In [ ]:
n_result = []
repetidos = []
for i in range(len(result)):
    for j in range(len(result)):
        if result[j][0]>=result[i][0] and result[j][1]>=result[i][1] and result[j][0]+result[j][2]<=result[i][0]+result[i][2] and result[j][1]+result[j][3]<=result[i][1]+result[i][3] and i!=j:
            repetidos.append(j)
            
for i in range(len(result)):
    if i not in repetidos:
        n_result.append(result[i])

In [ ]:
result

In [ ]:
n_result

In [ ]:
clone = b.copy()
print(len(n_result))
for i in range(len(n_result)):
    print(i)
    x,y,winW,winH = n_result[i]
    print(x,y,winW,winH)
    cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
    cv2.imshow("Window", clone)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

# Prediccion de las señales

Primero separamos las imagenes de señales en conjuntos de Train y Test como en el objetivo anterior para entrenar la SVM y para probarla. Y les aplicamos un filtro gaussiano el cual 

In [30]:
imagenesTrainForSignalRecog,imagenesTestForSignalRecog,ytrainForSignalRecog,ytestForSignalRecog = separaTrainTest(elem_signals,signals_types)

In [31]:
imagenesTrainForSignalRecog_GK = np.zeros(imagenesTrainForSignalRecog.shape)
for i in range(imagenesTrainForSignalRecog.shape[0]):
    imagenesTrainForSignalRecog_GK[i] = filtro_suavizado(imagenesTrainForSignalRecog[i], mask)

imagenesTestForSignalRecog_GK = np.zeros(imagenesTestForSignalRecog.shape)
for i in range(imagenesTestForSignalRecog_GK.shape[0]):
    imagenesTestForSignalRecog_GK[i] = filtro_suavizado(imagenesTestForSignalRecog[i], mask)
    
Xtrain_Sign, Ytrain_Sign = creaMatrizCaracteristicas(imagenesTrainForSignalRecog_GK,carac_type,ytrainForSignalRecog)
Xtest_Sign, Ytest_Sign = creaMatrizCaracteristicas(imagenesTestForSignalRecog_GK,carac_type,ytestForSignalRecog)

Entrenamos la SVM

In [32]:
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},]
svc = svm.SVC()
clf_signaltype = GridSearchCV(svc, param_grid)
clf_signaltype.fit(Xtrain_Sign, Ytrain_Sign.ravel())
y_pred = clf_signaltype.predict(Xtest_Sign)
print("Classification report for - \n{}:\n{}\n".format(clf_signaltype, metrics.classification_report(Ytest_Sign.ravel(), y_pred)))

C:\Users\daviz\anaconda3\envs\py37vision\lib\site-packages\sklearn\model_selection\_split.py:672: UserWarning: The least populated class in y has only 3 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


Classification report for - 
GridSearchCV(estimator=SVC(),
             param_grid=[{'C': [1, 10, 100, 1000], 'kernel': ['linear']},
                         {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001],
                          'kernel': ['rbf']}]):
                                         precision    recall  f1-score   support

                AdelantamientoProhibido       1.00      1.00      1.00        20
    AdelantamientoProhibidoParaCamiones       1.00      1.00      1.00        27
                    CalzadaConPrioridad       0.97      1.00      0.98        28
                             CedaElPaso       1.00      1.00      1.00        29
                   CirculacionProhibida       1.00      0.88      0.93         8
           CurvaPeligrosaHaciaLaDerecha       0.50      0.60      0.55         5
         CurvaPeligrosaHaciaLaIzquierda       0.00      0.00      0.00         3
       CurvasPeligrosasHaciaLaIzquierda       0.80      1.00      0.89         4
            

Probamos con el ejemplo anterio

In [33]:
#pequeña celda para interactuar con el clasificador

for i in range(len(n_result)):
    imagen_sin_color = a[n_result[i][1]:n_result[i][1] + n_result[i][3], n_result[i][0]:n_result[i][0] + n_result[i][2]]
    imagen_a_color = b[n_result[i][1]:n_result[i][1] + n_result[i][3], n_result[i][0]:n_result[i][0] + n_result[i][2]]
    res = predecirImagenSVM(cv2.resize(imagen_sin_color, (100, 100), interpolation = cv2.INTER_NEAREST),carac_type,clf_signaltype)
    plt.figure()
    plt.imshow(imagen_a_color)
    plt.text(imagen_sin_color.shape[1]-int(imagen_sin_color.shape[1]/2), imagen_sin_color.shape[0]+int(imagen_sin_color.shape[0]/7), res[0])
    plt.show()

NameError: name 'n_result' is not defined

In [ ]:
l = crearListaCalle()

In [ ]:
l[0].shape

In [34]:
def dondeHaySeñal(imagen_nombre,clf,clf_signaltype):
    imagen_sin_color = cv2.imread(imagen_nombre,0)
    imagen_color = cv2.imread(imagen_nombre)
    
    if imagen_sin_color.shape[0]<imagen_sin_color.shape[1]:
        winH = int(imagen_sin_color.shape[0])
    else:
        winH = int(imagen_sin_color.shape[1])
    winW = winH
    step = int(winW/2)
    result = []
    while winW>30 and winH>30:
        for (x, y, window) in sliding_window(imagen_sin_color, step, windowSize=(winW, winH)):
            if x+winW<=imagen_sin_color.shape[1] and y+winH<=imagen_sin_color.shape[0]:
                res = predecirImagenSVM(cv2.resize(window, (100, 100), interpolation = cv2.INTER_NEAREST),carac_type,clf)
                if res=='Signal':
                    result.append(np.array([x,y,winW,winH]))
        winW = int(winW/1.5)
        winH = winW
        step = int(winW/2)
        
#     si hay alguna prediccion incluida en otra se elimina la que está incluida en otra
    n_result = []
    repetidos = []
    for i in range(len(result)):
        for j in range(len(result)):
            if result[j][0]>=result[i][0] and result[j][1]>=result[i][1] and result[j][0]+result[j][2]<=result[i][0]+result[i][2] and result[j][1]+result[j][3]<=result[i][1]+result[i][3] and i!=j:
                repetidos.append(j)

    for i in range(len(result)):
        if i not in repetidos:
            n_result.append(result[i])
            
#     sacar la imagen con la señal identificada
    clone = imagen_color.copy()
    for i in range(len(n_result)):
        x,y,winW,winH = n_result[i]
        parche = imagen_sin_color[y:y + winH, x:x + winW]
        res = predecirImagenSVM(cv2.resize(parche, (100, 100), interpolation = cv2.INTER_NEAREST),carac_type,clf_signaltype)
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), (0, 255, 0), 2)
        cv2.putText(clone,str(res), (x,y-10), cv2.FONT_HERSHEY_PLAIN,.7,(0, 255, 0))
        cv2.imshow("Window", clone)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [ ]:
dondeHaySeñal('00058.jpg',clf,clf_signaltype)

In [ ]:
imagenes_calles_nombres = []
for i in range(0,100):
    if i<10:
        name = '0000'+str(i)+'.jpg'
    else:
        name = '000'+str(i)+'.jpg'
    imagenes_calles_nombres.append(name)

In [ ]:
imagenes_calles_nombres[0]

In [ ]:
dondeHaySeñal('00000.jpg',clf,clf_signaltype)